In [2]:
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
#import plotly.graph_objs as go
#import plotly.offline as py

In [3]:
# matplotlib 한글 출력..
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
path = 'C:/Windows/Fonts/malgun.ttf'  # 내가 원하는 폰트로 따로 설정이 가능
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

In [45]:
# MySQL Connection 연결
def player_name_call():
    conn = pymysql.connect(host='49.247.132.235', user='kbo_db', password='1234', db='kbo_data_db', charset='utf8')
    # Connection 으로부터 Cursor 생성
    curs = conn.cursor()
    sql = '''select * from kbo_pitcher_player 
            union
            select * from kbo_hitter_player;'''   
    curs.execute(sql)
    rows = curs.fetchall()
    df_player_name = pd.DataFrame(rows)
    conn.close()
    df_player_name.columns = ['player_id', 'player_name']
    return df_player_name

In [46]:
def MinMax_Scaler(df_scaler):
    scaler = MinMaxScaler()
    list_scaler_name = df_scaler['player_name'].values
    
    if 'mvp' in df_scaler.columns:
        list_scaler_mvp = df_scaler['mvp'].values
        df_scaler = df_scaler.drop(['player_name', 'mvp'], axis=1)
        df_scaler[df_scaler.columns] = scaler.fit_transform(df_scaler[df_scaler.columns])
        df_scaler['mvp'] = list_scaler_mvp
    else:       
        df_scaler = df_scaler.drop('player_name', axis=1)        
        df_scaler[df_scaler.columns] = scaler.fit_transform(df_scaler[df_scaler.columns])
        
    df_scaler['player_name'] = list_scaler_name
    
    return df_scaler

In [47]:
def hit_pit_sum (df_total_hit_m, df_total_pit_m):
    conn = pymysql.connect(host='49.247.132.235', user='kbo_db', password='1234', db='kbo_data_db', \
                           charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)
    curs = conn.cursor()
    sql = '''SELECT team_year, team_rank, team_name FROM kbo_team_rank where team_year = {};'''.format(int(df_total_hit_m.player_year[0]))
    curs.execute(sql)
    rows = curs.fetchall()
    team_year_rank = pd.DataFrame(rows)
    
    curs = conn.cursor()
    sql = '''SELECT player_year, player_name FROM kbo_mvp_player where player_year = {};'''.format(int(df_total_hit_m.player_year[0]))
    curs.execute(sql)
    rows = curs.fetchall()
    df_mvp_list = pd.DataFrame(rows)
    conn.close()    
    
    df_total_hit_m.columns = ['player_id', 'player_year', 'player_team', 'player_AVG', 'player_G', 'player_PA', 'player_AB', 'player_RH', 'player_HH',
       'player_2B', 'player_3B', 'player_HRH', 'player_TB', 'player_RBI', 'player_SB', 'player_CS', 'player_BBH', 
       'player_HBPH', 'player_SOH', 'player_GDP', 'player_SLG', 'player_OBP', 'player_E']
    df_total_pit_m.columns = ['player_id', 'player_year', 'player_team', 'player_ERA', 'player_GP', 'player_CG', 'player_SHO', 'player_W', 'player_L', 
       'player_SV', 'player_HLD', 'player_WPCT', 'player_TBF', 'player_IP', 'player_HP', 'player_HRP', 'player_BBP',
       'player_HBPP', 'player_SOP', 'player_RP', 'player_ER']
    df_sum_player = pd.merge(df_total_hit_m, df_total_pit_m, how='outer')
    
    df_player_name = player_name_call()
    df_sum_player = pd.merge(df_sum_player, df_player_name, on = 'player_id', how = 'left')
    df_sum_player = df_sum_player.fillna('0')
    
    df_sum_player['player_team'] = df_sum_player['player_team'].apply(lambda x: team_year_rank['team_rank'][x == team_year_rank['team_name']].values[0])
    df_sum_player = df_sum_player.drop(['player_id', 'player_year'], axis=1)
    df_sum_player = df_sum_player.apply(lambda x: x.replace('-','0'))
    df_sum_player['player_IP'] = df_sum_player['player_IP'].apply(lambda x: x.replace(' 1/3', '.3') if(' 1/3' in x) else (x.replace(' 2/3', '.6') if(' 2/3' in x) else x))
    df_sum_player['player_IP'] = df_sum_player['player_IP'].apply(lambda x: x.replace('1/3', '0.3') if('1/3' in x) else (x.replace('2/3', '0.6') if('2/3' in x) else x))

    for i in df_sum_player.columns:
        if not i in ['player_name']:
            df_sum_player[i] = df_sum_player[i].apply(lambda x: float(x))
            
    df_hit_m = df_sum_player.drop(list(df_sum_player[(df_sum_player.player_AVG == 0) & (df_sum_player.player_G == 0)].index))
    df_pit_m = df_sum_player[df_sum_player.player_AVG == 0]
            
    df_hit_m = MinMax_Scaler(df_hit_m)
    df_pit_m = MinMax_Scaler(df_pit_m)
    df_sum_player = pd.merge(df_hit_m, df_pit_m, how='outer')
    
    if df_total_hit_m.player_year[0] != '2020':
        print(df_total_hit_m.player_year[0], end='/')
        #df_mvp_list = pd.read_excel('./data/mvp_list.xlsx', index_col=0)
        df_sum_player['mvp'] = df_sum_player['player_name'].apply(lambda x: 1 if (df_mvp_list.player_name[0] == x) else 0)
   # df_sum_player = df_sum_player.drop(['player_PA', 'player_TB', 'player_SLG', 'player_OBP', 'player_E', 'player_CG', 'player_SHO'], axis=1)
    #df_sum_player[['player_team','player_GDP','player_SOH','player_CS','player_ERA', 'player_L', 'player_HP', 'player_HRP', 'player_BBP', 'player_HBPP', 'player_RP', 'player_ER']] = \
    #df_sum_player[['player_team','player_GDP','player_SOH','player_CS','player_ERA', 'player_L', 'player_HP', 'player_HRP', 'player_BBP', 'player_HBPP', 'player_RP', 'player_ER']].apply(lambda x: round((1-x), 6))
    return df_sum_player

In [48]:
def make_team_df(data_year):
    conn = pymysql.connect(host='49.247.132.235', user='kbo_db', password='1234', db='kbo_data_db', \
                           charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)
    curs = conn.cursor()
    #sql = '''SELECT * from kbo_hitter_total_status where player_G > 50 and player_year = {};'''.format(data_year)   
    sql = '''select x.* from (SELECT * from kbo_hitter_total_status where player_year = {}
        and player_PA > 100 
        order by player_PA desc) x
        limit 75;'''.format(data_year)
    curs.execute(sql)
    rows = curs.fetchall()
    df_year_hit = pd.DataFrame(rows)

    curs = conn.cursor()
    #sql = '''SELECT * from kbo_pitcher_total_status where player_IP > 120 and player_year = {};'''.format(data_year)    
    sql = '''select x.* from (SELECT * from kbo_pitcher_total_status where player_year = {}
        and player_IP > 100 
        order by player_IP desc) x
        limit 25;'''.format(data_year)
    curs.execute(sql)
    rows = curs.fetchall()
    df_year_pit = pd.DataFrame(rows)
    conn.close()
    df_year_player = hit_pit_sum(df_year_hit, df_year_pit)
    return df_year_player

In [49]:
def df_train_data (data_year):
    df_train_player = pd.DataFrame()
    for i in range(2001, 2020):
        if i != data_year:
            df_year_player = make_team_df(i)
            df_train_player = pd.concat([df_train_player, df_year_player])
    df_test_player = make_team_df(data_year)
    return df_train_player, df_test_player

In [50]:
df_trian_2019, df_test_2019 = df_train_data(2020)

2001/2002/2003/2004/2005/2006/2007/2008/2009/2010/2011/2012/2013/2014/2015/2016/2017/2018/2019/

,player_team,player_AVG,player_G,player_PA,player_AB,player_RH,player_HH,player_2B,player_3B,player_HRH,...,player_IP,player_HP,player_HRP,player_BBP,player_HBPP,player_SOP,player_RP,player_ER,player_name,mvp
0,0.714286,0.661871,1.000000,1.000000,1.000000,1.000000,1.000000,0.884615,0.444444,0.307692,...,0.000000,0.00,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,이병규,0
1,0.571429,0.683453,1.000000,0.955556,0.837423,0.764045,0.868421,0.961538,0.444444,0.589744,...,0.000000,0.00,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,장성호,0
2,0.714286,0.482014,0.940299,0.925000,0.711656,0.808989,0.649123,0.730769,0.222222,0.230769,...,0.000000,0.00,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,유지현,0
3,0.142857,0.431655,0.910448,0.908333,0.757669,0.932584,0.657895,0.923077,0.222222,1.000000,...,0.000000,0.00,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,이승엽,1
4,0.285714,0.489209,0.910448,0.897222,0.837423,0.640449,0.754386,0.500000,0.333333,0.461538,...,0.000000,0.00,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,박재홍,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.162272,0.62,0.6250,0.815789,1.000000,0.456,1.000000,1.000000,터너,0
96,0.555556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.107505,0.54,0.8750,1.000000,0.133333,0.216,0.660714,0.782609,김민,0
97,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.081136,0.32,0.8125,0.342105,0.266667,0.424,0.535714,0.543478,다익손,0
98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.060852,0.40,0.8125,0.473684,0.266667,0.304,0.428571,0.434783,이용찬,0
